In [3]:
pip install pandas numpy python-dateutil sqlalchemy


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dateutil.parser import parse
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(42)


In [5]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dateutil.parser import parse
from sqlalchemy import create_engine

np.random.seed(42)


In [6]:
df = pd.read_csv('/Users/muhammadarslanarshad/Downloads/ML projects datasets/E-commerce/ecommerce_dataset_updated.csv')

# Quick look at the data
print(df.head())
print(df.info())
print(df.shape)

    User_ID  Product_ID  Category  Price (Rs.)  Discount (%)  \
0  337c166f  f414122f-e    Sports        36.53            15   
1  d38a19bf  fde50f9c-5  Clothing       232.79            20   
2  d7f5f0b0  0d96fc90-3    Sports       317.02            25   
3  395d4994  964fc44b-d      Toys       173.19            25   
4  a83c145c  d70e2fc6-e    Beauty       244.80            20   

   Final_Price(Rs.) Payment_Method Purchase_Date  
0             31.05    Net Banking    12-11-2024  
1            186.23    Net Banking    09-02-2024  
2            237.76    Credit Card    01-09-2024  
3            129.89            UPI    01-04-2024  
4            195.84    Net Banking    27-09-2024  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3660 entries, 0 to 3659
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   User_ID           3660 non-null   object 
 1   Product_ID        3660 non-null   object 
 2   Category  

In [10]:
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Decide number of products per order (simulate 1-4)
num_products = np.random.choice([1, 2, 3, 4], size=len(df), p=[0.5, 0.3, 0.15, 0.05])

order_id_list = []
transaction_id_list = []
current_order = 1000
current_transaction = 5000

i = 0
while i < len(df):
    n = num_products[i]
    for j in range(n):
        if i + j < len(df):
            order_id_list.append(current_order)
            transaction_id_list.append(current_transaction)
    current_order += 1
    current_transaction += 1
    i += n

# Add to df
df = df.iloc[:len(order_id_list)]  # trim if extra
df['order_id'] = order_id_list
df['transaction_id'] = transaction_id_list

In [11]:
# Aggregate order-level info
orders = df.groupby('order_id').agg({
    'transaction_id': 'first',
    'User_ID': 'first',
    'Payment_Method': 'first',
}).reset_index()

# Add shipping_cost per order (random realistic)
orders['shipping_cost'] = np.round(np.random.uniform(5, 20, len(orders)), 2)

# Add order_status
orders['order_status'] = np.random.choice(['Delivered','Cancelled','Returned'], size=len(orders), p=[0.85,0.10,0.05])

# Calculate order_total from product-level data
df['final_price'] = df['Final_Price(Rs.)']
order_totals = df.groupby('order_id')['final_price'].sum().reset_index()
order_totals.rename(columns={'final_price':'order_total'}, inplace=True)

orders = orders.merge(order_totals, on='order_id', how='left')


In [12]:
# Example: pick a random order
sample_order_id = orders['order_id'].sample(1).values[0]

print("Order in Orders Table:")
print(orders[orders['order_id']==sample_order_id])

print("\nAll Products in this Order:")
print(df[df['order_id']==sample_order_id])

# Check sum of final_price matches order_total
calculated_total = df[df['order_id']==sample_order_id]['final_price'].sum()
order_total = orders[orders['order_id']==sample_order_id]['order_total'].values[0]

print("\nCheck totals match:", calculated_total == order_total)


Order in Orders Table:
     order_id  transaction_id   User_ID Payment_Method  shipping_cost  \
450      1450            5450  3f68ed08            UPI           7.89   

    order_status  order_total  
450    Delivered       257.56  

All Products in this Order:
      User_ID  Product_ID        Category  Price (Rs.)  Discount (%)  \
751  3f68ed08  f01054be-4  Home & Kitchen       367.95            30   

     Final_Price(Rs.) Payment_Method Purchase_Date  order_id  transaction_id  \
751            257.56            UPI    06-05-2024      1450            5450   

     final_price  
751       257.56  

Check totals match: True


In [13]:
df.to_csv('ecommerce_dataset_updated.csv', index=False)
orders.to_csv('orders_table.csv', index=False)


In [17]:
duplicate_orders = df[df.duplicated(subset=['order_id'], keep=False)]

# Number of duplicate order_ids
num_duplicates = duplicate_orders['order_id'].nunique()
print("Number of order_ids that appear more than once:", num_duplicates)

# Preview some duplicates
print("\nSample duplicate order_ids and their products:")
print(duplicate_orders.sort_values('order_id').head(20))

Number of order_ids that appear more than once: 1036

Sample duplicate order_ids and their products:
     User_ID  Product_ID        Category  Price (Rs.)  Discount (%)  \
1   133c23df  1676300b-1            Toys       233.53            25   
2   09b8e30c  d57ebdfe-f            Toys        71.25             0   
3   526f0441  a2638fa2-0            Toys        14.98            50   
4   91932784  1306885a-f           Books       198.11            50   
7   8c3265d7  08562850-3          Sports       269.87            50   
8   839f23dc  2838f818-c  Home & Kitchen       285.47            20   
9   847a300e  233f387c-1        Clothing       103.44            30   
10  1dac4ff2  ebe13d4f-7          Sports       433.94             0   
13  29b418a7  741436e6-7          Sports       468.72            30   
11  95912e32  9bb22a8b-a  Home & Kitchen        92.03            20   
12  8b962cd8  5ed1a6ec-f  Home & Kitchen       192.28            20   
18  f01b43f7  5e6ccb0a-7        Clothing       

In [18]:
# Quantity per product (simulate)
df['quantity'] = np.random.choice([1,2,3,4], size=len(df), p=[0.6,0.25,0.1,0.05])

# Customer country
df['customer_country'] = np.random.choice(['India','UAE','UK','USA','Others'], size=len(df), p=[0.7,0.1,0.08,0.05,0.07])

# Customer age group
df['customer_age_group'] = np.random.choice(['18-25','26-35','36-50','50+'], size=len(df), p=[0.25,0.4,0.25,0.1])

# Customer signup date (before purchase)
df['customer_signup_date'] = pd.to_datetime(df['Purchase_Date']) - pd.to_timedelta(np.random.randint(30, 1000, size=len(df)), unit='D')

# Product brand
df['brand'] = np.random.choice(['Brand A','Brand B','Brand C','Brand D'], size=len(df))

# Price band
def price_band(price):
    if price < 50:
        return 'Low'
    elif price < 200:
        return 'Medium'
    else:
        return 'High'

df['price_band'] = df['Price (Rs.)'].apply(price_band)


/var/folders/tn/nx0fp4655xs3pdpnrh5z4yt80000gn/T/ipykernel_1874/1706949929.py:11: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['customer_signup_date'] = pd.to_datetime(df['Purchase_Date']) - pd.to_timedelta(np.random.randint(30, 1000, size=len(df)), unit='D')


In [19]:
# Calculate order_total
df['net_revenue'] = df['Final_Price(Rs.)'] * df['quantity']
order_totals = df.groupby('order_id')['net_revenue'].sum().reset_index().rename(columns={'net_revenue':'order_total'})
orders = orders.merge(order_totals, on='order_id', how='left')

# Total items per order
order_items_count = df.groupby('order_id')['Product_ID'].count().reset_index().rename(columns={'Product_ID':'total_items'})
orders = orders.merge(order_items_count, on='order_id', how='left')


In [21]:
df.to_csv('ecommerce_dataset_updated.csv', index=False)
orders.to_csv('orders_table.csv', index=False)


In [ ]:

orders.head()

,order_id,transaction_id,User_ID,Payment_Method,shipping_cost,order_status,total_items,order_total
0,1000,5000,14a322ad,UPI,15.39,Delivered,1,260.06
1,1001,5001,133c23df,Debit Card,13.88,Delivered,2,596.70
2,1002,5002,526f0441,Net Banking,18.18,Delivered,2,114.04
3,1003,5003,391837b0,Debit Card,10.35,Cancelled,1,836.96
4,1004,5004,a8fc7be9,Net Banking,9.50,Returned,1,46.20


In [26]:
df.head()

,User_ID,Product_ID,Category,Price (Rs.),Discount (%),Final_Price(Rs.),Payment_Method,Purchase_Date,order_id,transaction_id,final_price,quantity,customer_country,customer_age_group,customer_signup_date,brand,price_band,net_revenue
0,14a322ad,880eab94-7,Home & Kitchen,152.98,15,130.03,UPI,18-06-2024,1000,5000,130.03,2,India,50+,2022-10-04,Brand D,Medium,260.06
1,133c23df,1676300b-1,Toys,233.53,25,175.15,Debit Card,09-11-2024,1001,5001,175.15,3,India,26-35,2023-10-12,Brand C,High,525.45
2,09b8e30c,d57ebdfe-f,Toys,71.25,0,71.25,UPI,14-03-2024,1001,5001,71.25,1,India,18-25,2022-02-06,Brand C,Medium,71.25
3,526f0441,a2638fa2-0,Toys,14.98,50,7.49,Net Banking,30-09-2024,1002,5002,7.49,2,India,36-50,2023-06-30,Brand A,Low,14.98
4,91932784,1306885a-f,Books,198.11,50,99.06,Credit Card,22-07-2024,1002,5002,99.06,1,India,26-35,2024-01-23,Brand D,Medium,99.06


In [33]:
import pandas as pd

# Load your tables
orders = pd.read_csv('orders_table.csv')
df = pd.read_csv('ecommerce_dataset_updated.csv')

# Ensure quantity and Final_Price(Rs.) are numeric
df['quantity'] = pd.to_numeric(df['quantity'], errors='coerce')
df['Final_Price(Rs.)'] = pd.to_numeric(df['Final_Price(Rs.)'], errors='coerce')

# 1️⃣ Make sure transaction_id exists in orders
if 'transaction_id' not in orders.columns:
    orders['transaction_id'] = range(5000, 5000 + len(orders))

# 2️⃣ Calculate net revenue per product
df['net_revenue'] = df['Final_Price(Rs.)'] * df['quantity']

# 3️⃣ Aggregate to get order_total per order
order_totals = df.groupby('order_id')['net_revenue'].sum().reset_index()
order_totals.rename(columns={'net_revenue':'order_total'}, inplace=True)

# 4️⃣ Merge order totals into orders table
orders = orders.drop(columns=['order_total'], errors='ignore')  # remove any old column
orders = orders.merge(order_totals, on='order_id', how='left')

# 5️⃣ Save back to the same CSV
orders.to_csv('orders_table.csv', index=False)

# 6️⃣ Preview
print(orders.head())


   order_id  transaction_id   User_ID Payment_Method  shipping_cost  \
0      1000            5000  14a322ad            UPI          15.39   
1      1001            5001  133c23df     Debit Card          13.88   
2      1002            5002  526f0441    Net Banking          18.18   
3      1003            5003  391837b0     Debit Card          10.35   
4      1004            5004  a8fc7be9    Net Banking           9.50   

  order_status  total_items  order_total  
0    Delivered            1       260.06  
1    Delivered            2       596.70  
2    Delivered            2       114.04  
3    Cancelled            1       836.96  
4     Returned            1        46.20  


In [36]:
# If 'transaction_id' does not exist or has missing values, create unique ones
if 'transaction_id' not in orders.columns or orders['transaction_id'].isnull().any():
    orders = orders.drop(columns=['transaction_id'], errors='ignore')
    orders['transaction_id'] = range(5000, 5000 + len(orders))

# Check uniqueness
assert orders['order_id'].nunique() == orders['transaction_id'].nunique(), "Each order_id must have a unique transaction_id"


In [37]:
# Ensure numeric columns
df['quantity'] = pd.to_numeric(df['quantity'], errors='coerce')
df['Final_Price(Rs.)'] = pd.to_numeric(df['Final_Price(Rs.)'], errors='coerce')

# Calculate net revenue per product
df['net_revenue'] = df['Final_Price(Rs.)'] * df['quantity']

# Aggregate by order_id to get total revenue per order
order_totals = df.groupby('order_id')['net_revenue'].sum().reset_index()
order_totals.rename(columns={'net_revenue':'order_total'}, inplace=True)

# Merge into orders table
orders = orders.drop(columns=['order_total'], errors='ignore')  # remove old columns
orders = orders.merge(order_totals, on='order_id', how='left')
# Ensure numeric columns
df['quantity'] = pd.to_numeric(df['quantity'], errors='coerce')
df['Final_Price(Rs.)'] = pd.to_numeric(df['Final_Price(Rs.)'], errors='coerce')

# Calculate net revenue per product
df['net_revenue'] = df['Final_Price(Rs.)'] * df['quantity']

# Aggregate by order_id to get total revenue per order
order_totals = df.groupby('order_id')['net_revenue'].sum().reset_index()
order_totals.rename(columns={'net_revenue':'order_total'}, inplace=True)

# Merge into orders table
orders = orders.drop(columns=['order_total'], errors='ignore')  # remove old columns
orders = orders.merge(order_totals, on='order_id', how='left')


In [38]:
dim_customer = df[['User_ID','customer_country','customer_age_group','customer_signup_date']].drop_duplicates().reset_index(drop=True)
dim_product = df[['Product_ID','Category','brand','price_band']].drop_duplicates().reset_index(drop=True)
date_dim_df = df[['Purchase_Date']].drop_duplicates().reset_index(drop=True)
date_dim_df['date_id'] = range(1, len(date_dim_df)+1)
date_dim_df['year'] = date_dim_df['Purchase_Date'].dt.year
date_dim_df['month'] = date_dim_df['Purchase_Date'].dt.month
date_dim_df['day'] = date_dim_df['Purchase_Date'].dt.day
date_dim_df['quarter'] = date_dim_df['Purchase_Date'].dt.quarter
date_dim_df['day_of_week'] = date_dim_df['Purchase_Date'].dt.day_name()
dim_date = date_dim_df[['date_id','Purchase_Date','year','month','day','quarter','day_of_week']]
dim_payment = df[['Payment_Method']].drop_duplicates().reset_index(drop=True)
dim_payment['payment_id'] = range(1, len(dim_payment)+1)


In [39]:
# -----------------------------
# Step 4b – Save Dimension Tables
# -----------------------------

# Customer Dimension
dim_customer.to_csv('dim_customer.csv', index=False)

# Product Dimension
dim_product.to_csv('dim_product.csv', index=False)

# Date Dimension
dim_date.to_csv('dim_date.csv', index=False)

# Payment Dimension
dim_payment.to_csv('dim_payment.csv', index=False)

# Preview to confirm
print("Customer Dimension:")
print(dim_customer.head())
print("\nProduct Dimension:")
print(dim_product.head())
print("\nDate Dimension:")
print(dim_date.head())
print("\nPayment Dimension:")
print(dim_payment.head())


Customer Dimension:
    User_ID customer_country customer_age_group customer_signup_date
0  14a322ad            India                50+           2022-10-04
1  133c23df            India              26-35           2023-10-12
2  09b8e30c            India              18-25           2022-02-06
3  526f0441            India              36-50           2023-06-30
4  91932784            India              26-35           2024-01-23

Product Dimension:
   Product_ID        Category    brand price_band
0  880eab94-7  Home & Kitchen  Brand D     Medium
1  1676300b-1            Toys  Brand C       High
2  d57ebdfe-f            Toys  Brand C     Medium
3  a2638fa2-0            Toys  Brand A        Low
4  1306885a-f           Books  Brand D     Medium

Date Dimension:
   date_id Purchase_Date  year  month  day  quarter day_of_week
0        1    2024-06-18  2024      6   18        2     Tuesday
1        2    2024-11-09  2024     11    9        4    Saturday
2        3    2024-03-14  2024      3

In [41]:
orders.head()

,order_id,transaction_id,User_ID,Payment_Method,shipping_cost,order_status,total_items,Purchase_Date,order_total
0,1000,5000,14a322ad,UPI,15.39,Delivered,1,NaT,260.06
1,1001,5001,133c23df,Debit Card,13.88,Delivered,2,NaT,596.70
2,1002,5002,526f0441,Net Banking,18.18,Delivered,2,NaT,114.04
3,1003,5003,391837b0,Debit Card,10.35,Cancelled,1,NaT,836.96
4,1004,5004,a8fc7be9,Net Banking,9.50,Returned,1,NaT,46.20


In [42]:
# Load date dimension
dim_date = pd.read_csv('dim_date.csv')

# Preview
print(dim_date.head())

# Check if orders' purchase dates exist in date dimension
print("Purchase_Date column in orders exists? ->", 'Purchase_Date' in orders.columns)


   date_id Purchase_Date  year  month  day  quarter day_of_week
0        1    2024-06-18  2024      6   18        2     Tuesday
1        2    2024-11-09  2024     11    9        4    Saturday
2        3    2024-03-14  2024      3   14        1    Thursday
3        4    2024-09-30  2024      9   30        3      Monday
4        5    2024-07-22  2024      7   22        3      Monday
Purchase_Date column in orders exists? -> True


In [46]:
import pandas as pd
import numpy as np

# -----------------------------
# 1️⃣ Load tables
# -----------------------------
df = pd.read_csv('ecommerce_dataset_updated.csv')
orders = pd.read_csv('orders_table.csv')

# Ensure dates are datetime
df['Purchase_Date'] = pd.to_datetime(df['Purchase_Date'], errors='coerce')

# -----------------------------
# 2️⃣ Prepare Orders Table
# -----------------------------
# Ensure transaction_id exists and is unique
if 'transaction_id' not in orders.columns or orders['transaction_id'].isnull().any():
    orders['transaction_id'] = range(5000, 5000 + len(orders))

# Ensure numeric columns
df['quantity'] = pd.to_numeric(df['quantity'], errors='coerce')
df['Final_Price(Rs.)'] = pd.to_numeric(df['Final_Price(Rs.)'], errors='coerce')

# Calculate net revenue per product
df['net_revenue'] = df['Final_Price(Rs.)'] * df['quantity']

# Calculate order_total per order
order_totals = df.groupby('order_id')['net_revenue'].sum().reset_index()
order_totals.rename(columns={'net_revenue':'order_total'}, inplace=True)
orders = orders.drop(columns=['order_total'], errors='ignore')
orders = orders.merge(order_totals, on='order_id', how='left')

# Save orders table
orders.to_csv('fact_orders.csv', index=False)

# -----------------------------
# 3️⃣ Create Dimension Tables
# -----------------------------
dim_customer = df[['User_ID','customer_country','customer_age_group','customer_signup_date']].drop_duplicates().reset_index(drop=True)
dim_customer.to_csv('dim_customer.csv', index=False)

dim_product = df[['Product_ID','Category','brand','price_band']].drop_duplicates().reset_index(drop=True)
dim_product.to_csv('dim_product.csv', index=False)

dim_payment = df[['Payment_Method']].drop_duplicates().reset_index(drop=True)
dim_payment['payment_id'] = range(1, len(dim_payment)+1)
dim_payment.to_csv('dim_payment.csv', index=False)

dim_date = df[['Purchase_Date']].drop_duplicates().reset_index(drop=True)
dim_date['date_id'] = range(1, len(dim_date)+1)
dim_date['year'] = dim_date['Purchase_Date'].dt.year
dim_date['month'] = dim_date['Purchase_Date'].dt.month
dim_date['day'] = dim_date['Purchase_Date'].dt.day
dim_date['quarter'] = dim_date['Purchase_Date'].dt.quarter
dim_date['day_of_week'] = dim_date['Purchase_Date'].dt.day_name()
dim_date.to_csv('dim_date.csv', index=False)

# -----------------------------
# 4️⃣ Build Fact Table
# -----------------------------
fact_sales = df.copy()

# Merge order info
fact_sales = fact_sales.merge(
    orders[['order_id','transaction_id','shipping_cost','order_status','order_total']],
    on='order_id',
    how='left'
)

# Handle possible _x/_y duplicates
if 'transaction_id_y' in fact_sales.columns:
    fact_sales = fact_sales.rename(columns={'transaction_id_y':'transaction_id'})
fact_sales = fact_sales.drop(columns=['transaction_id_x'], errors='ignore')

# Merge dimension tables
fact_sales = fact_sales.merge(dim_customer, on='User_ID', how='left')
fact_sales = fact_sales.merge(dim_product, on='Product_ID', how='left')
fact_sales = fact_sales.merge(dim_payment, on='Payment_Method', how='left')
fact_sales = fact_sales.merge(dim_date[['Purchase_Date','date_id']], on='Purchase_Date', how='left')

# Select only existing columns to avoid KeyError
columns_needed = ['order_id','transaction_id','User_ID','Product_ID','quantity','Final_Price(Rs.)',
                  'net_revenue','date_id','Payment_Method','payment_id','shipping_cost',
                  'order_status','order_total']
columns_to_keep = [c for c in columns_needed if c in fact_sales.columns]
fact_sales = fact_sales[columns_to_keep]

# Save fact table
fact_sales.to_csv('fact_sales.csv', index=False)

# -----------------------------
# 5️⃣ Preview tables
# -----------------------------
print("Fact Sales Table:")
print(fact_sales.head())
print("\nCustomer Dimension:")
print(dim_customer.head())
print("\nProduct Dimension:")
print(dim_product.head())
print("\nPayment Dimension:")
print(dim_payment.head())
print("\nDate Dimension:")
print(dim_date.head())
print("\nOrders Table:")
print(orders.head())


Fact Sales Table:
   order_id  transaction_id   User_ID  Product_ID  quantity  Final_Price(Rs.)  \
0      1000            5000  14a322ad  880eab94-7         2            130.03   
1      1001            5001  133c23df  1676300b-1         3            175.15   
2      1001            5001  09b8e30c  d57ebdfe-f         1             71.25   
3      1002            5002  526f0441  a2638fa2-0         2              7.49   
4      1002            5002  91932784  1306885a-f         1             99.06   

   net_revenue  date_id Payment_Method  payment_id  shipping_cost  \
0       260.06        1            UPI           1          15.39   
1       525.45        2     Debit Card           2          13.88   
2        71.25        3            UPI           1          13.88   
3        14.98        4    Net Banking           3          18.18   
4        99.06        5    Credit Card           4          18.18   

  order_status  order_total  
0    Delivered       260.06  
1    Delivered      

/var/folders/tn/nx0fp4655xs3pdpnrh5z4yt80000gn/T/ipykernel_1874/305781675.py:11: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Purchase_Date'] = pd.to_datetime(df['Purchase_Date'], errors='coerce')


In [53]:
import pandas as pd
from sqlalchemy import create_engine, text

# -----------------------------
# 1️⃣ Load CSVs
# -----------------------------
dim_customer = pd.read_csv('dim_customer.csv')
dim_product = pd.read_csv('dim_product.csv')
dim_payment = pd.read_csv('dim_payment.csv')
dim_date = pd.read_csv('dim_date.csv')
fact_sales = pd.read_csv('fact_sales.csv')
fact_orders = pd.read_csv('fact_orders.csv')

# -----------------------------
# 2️⃣ Connect to PostgreSQL
# -----------------------------
user = 'postgres'
password = 'X0000'
host = 'localhost'
port = '5432'
database = 'ecommerce'

engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

# -----------------------------
# 3️⃣ Load Dimension Tables
# -----------------------------
dim_customer.to_sql('dim_customer', engine, if_exists='replace', index=False)
dim_product.to_sql('dim_product', engine, if_exists='replace', index=False)
dim_payment.to_sql('dim_payment', engine, if_exists='replace', index=False)
dim_date.to_sql('dim_date', engine, if_exists='replace', index=False)

# -----------------------------
# 4️⃣ Load Fact Tables
# -----------------------------
fact_orders.to_sql('fact_orders', engine, if_exists='replace', index=False)
fact_sales.to_sql('fact_sales', engine, if_exists='replace', index=False)

# -----------------------------
# 5️⃣ Confirm Load
# -----------------------------
with engine.connect() as conn:
    result = conn.execute(text("SELECT COUNT(*) FROM fact_sales;"))
    print("Rows in fact_sales:", result.scalar())
    
    result = conn.execute(text("SELECT COUNT(*) FROM fact_orders;"))
    print("Rows in fact_orders:", result.scalar())


InternalError: (psycopg2.errors.DependentObjectsStillExist) cannot drop table dim_customer because other objects depend on it
DETAIL:  constraint fk_user on table fact_sales depends on table dim_customer
constraint fk_user_orders on table fact_orders depends on table dim_customer
HINT:  Use DROP ... CASCADE to drop the dependent objects too.

[SQL: 
DROP TABLE dim_customer]
(Background on this error at: https://sqlalche.me/e/20/2j85)

In [ ]:
df.columns

Index(['User_ID', 'Product_ID', 'Category', 'Price (Rs.)', 'Discount (%)',
       'Final_Price(Rs.)', 'Payment_Method', 'Purchase_Date', 'order_id',
       'transaction_id', 'final_price', 'quantity', 'customer_country',
       'customer_age_group', 'customer_signup_date', 'brand', 'price_band',
       'net_revenue'],
      dtype='object')

In [ ]:
# Merge original price from df into fact_sales
fact_sales = fact_sales.merge(
    df[['order_id', 'Product_ID', 'Price (Rs.)']],
    on=['order_id', 'Product_ID'],
    how='left'
)

# Calculate discount percentage
fact_sales['Discount_Percent'] = round(
    ((fact_sales['Price (Rs.)'] - fact_sales['Final_Price(Rs.)']) / fact_sales['Price (Rs.)']) * 100, 2
)


In [ ]:
fact_sales.to_csv('fact_sales.csv', index=False)


In [54]:


# -----------------------------
# 1. Load original fact_sales CSV
# -----------------------------
fact_sales = pd.read_csv('fact_sales.csv')

# -----------------------------
# 2. PostgreSQL connection
# -----------------------------
host = "localhost"
port = 5432
database = "ecommerce"
user = "postgres"
password = "X0000"

engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

# -----------------------------
# 3. Load CSV into PostgreSQL
# -----------------------------
# Table will be created as "fact_sales" in the database
fact_sales.to_sql(
    name='fact_sales',   # table name in PostgreSQL
    con=engine,
    if_exists='replace', # overwrite if table exists
    index=False
)

print("Original fact_sales loaded successfully into PostgreSQL!")


Original fact_sales loaded successfully into PostgreSQL!
